In [10]:
import pandas as pd
import numpy as np
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, XLMRobertaTokenizer

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

c:\Users\Tanner\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
print(torch.cuda.is_available())

True


In [4]:
model_2 = BertForSequenceClassification.from_pretrained("IndustryModelv1")
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
cap_df = pd.read_csv("C:/Users/Tanner/OneDrive/Documents/School_Work/Capstone/2024-04-24_zig_analytics.csv")
cap_df.head()

In [5]:
inputs = "Un restaurante que sirve carne de res y pollo."

In [7]:
inputs = tokenizer(inputs,padding = True, truncation = True, return_tensors='pt').to('cuda')
print(inputs)

{'input_ids': tensor([[  101,  4895,  4825,  2063, 10861,  2909,  3726,  2482,  2638,  2139,
         24501,  1061,  8554,  2080,  1012,   102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [8]:
def predict_in_batches(model, inputs, batch_size=32):
    model.eval()
    all_logits = []
    for i in range(0, len(inputs['input_ids']), batch_size):
        batch = {k: v[i:i+batch_size].to(model.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**batch)
            logits = outputs.logits
        all_logits.append(logits.cpu())
    return torch.cat(all_logits)

batch_size = 8
logits = predict_in_batches(model_2, inputs, batch_size)
predicted_labels = torch.argmax(logits, dim=1).tolist()

In [9]:
print(predicted_labels)

[11]
